In [27]:
from selenium import webdriver
driver=webdriver.Firefox()
driver.implicitly_wait(3)
driver.get("https://www.fundsexplorer.com.br/ranking")

In [28]:
from time import sleep
from selenium.webdriver.common.by import By
fecha_aceitar = '//*[@id="hs-eu-confirmation-button"]'#"/html/body/div[2]/div/div[2]/div/a"
sleep(10)
driver.find_element(By.XPATH,fecha_aceitar).click()

In [29]:
#-------------# para apagar propaganda popup
try:
    #-------------# Muda o contexto para o iframe 
    iframe_element = driver.find_element(By.XPATH, "//iframe[@title='Popup CTA']")
    driver.switch_to.frame(iframe_element)
    # Encontra o botão "X" dentro do iframe
    close_button = driver.find_element(By.XPATH, "/html/body/div/div[1]")
    close_button.click()
    # Volte para o contexto principal
    driver.switch_to.default_content()
except:
    print('não achei iframe')
    print('FECHE NA MÃO')
    sleep(5)


não achei iframe
FECHE NA MÃO


In [30]:
import pandas as pd
import numpy as np
from datetime import date
today = date.today().strftime('%d/%m/%Y')

In [31]:
def limpar_dados_para_json(df):
    """
    Limpa o DataFrame removendo valores que não são compatíveis com JSON
    """
    # Fazer uma cópia para não modificar o original
    df_clean = df.copy()
    
    # Substituir valores infinitos e NaN por strings ou valores válidos
    df_clean = df_clean.replace([np.inf, -np.inf], 'N/A')
    df_clean = df_clean.fillna('N/A')
    
    # Converter todas as colunas para string para evitar problemas de tipo
    for col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str)
    
    return df_clean



print(" ====== Rola pagina")

# Scroll up the window by a specific number of pixels
# For example, scroll up by 200 pixels
scroll_distance = 500
# print('111')
driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
# print('222')
sleep(2)

print(" ====== Escolhe todas as colunas ")

driver.find_element(By.XPATH,'//*[@id="colunas-ranking__select-button"]').click()
sleep(2)
driver.find_element(By.XPATH,'/html/body/div[7]/div[1]/div/div[2]/div[2]/ul/li[1]/label').click()
sleep(2)

print(" ====== lê a tabela ")

from io import StringIO

html_str = driver.page_source
tabelas_html = pd.read_html(StringIO(html_str))

df=tabelas_html[0]

driver.close()
# Limpar dados antes de retornar
df = limpar_dados_para_json(df)



 ====== Rola pagina
 ====== Escolhe todas as colunas 
 ====== lê a tabela 


In [32]:
def converter_valor(x):
    if pd.isna(x) or x == 'N/A':
        return np.nan
    x = str(x)
    if ',' in x:  # formato brasileiro
        x = x.replace('.', '').replace(',', '.')
        return float(x)
    else:  # sem vírgula → interpretar como centavos implícitos
        return float(x) / 100

df['Preço Atual (R$)'] = df['Preço Atual (R$)'].apply(converter_valor).apply(lambda x: str(x).replace('.', ','))
df['Último Dividendo'] = df['Último Dividendo'].apply(converter_valor).apply(lambda x: str(x).replace('.', ','))
df['VPA'] = df['VPA'].apply(converter_valor).apply(lambda x: str(x).replace('.', ','))


In [40]:
df['P/VP'].unique()

array(['N/A', '86.0', '78.0', '98.0', '76.0', '60.0', '62.0', '29.0',
       '80.0', '85.0', '94.0', '68.0', '87.0', '105.0', '79.0', '41.0',
       '112.0', '64.0', '93.0', '89.0', '74.0', '77.0', '84.0', '70.0',
       '143.0', '134.0', '46.0', '11.0', '90.0', '100.0', '92.0', '91.0',
       '63.0', '49.0', '165.0', '111.0', '160.0', '69.0', '138.0', '83.0',
       '96.0', '48.0', '99.0', '15.0', '36.0', '145.0', '101.0', '59.0',
       '103.0', '66.0', '23.0', '17.0', '95.0', '123.0', '82.0', '104.0',
       '67.0', '58.0', '65.0', '56.0', '32.0', '72.0', '18.0', '1453.0',
       '10.0', '61.0', '107.0', '73.0', '102.0', '43.0', '109.0', '71.0',
       '2382.0', '13.0', '81.0', '22.0', '88.0', '97.0', '42.0', '113.0',
       '106.0', '125.0', '1667.0', '195.0', '153.0', '118.0', '16.0',
       '4.0', '110.0', '3051.0', '178.0', '763.0', '115.0', '55.0',
       '136.0', '189.0', '14.0', '141.0', '119.0', '132.0', '313.0',
       '139.0', '142.0', '23511.0', '133.0', '122.0', '45.0', 

In [34]:
df

,Fundos,Setor,Preço Atual (R$),Liquidez Diária (R$),P/VP,Último Dividendo,Dividend Yield,DY (3M) Acumulado,DY (6M) Acumulado,DY (12M) Acumulado,...,DY Patrimonial,Variação Patrimonial,Rentab. Patr. Período,Rentab. Patr. Acumulada,Quant. Ativos,Volatilidade,Num. Cotistas,Tax. Gestão,Tax. Performance,Tax. Administração
0,AAGR11,Indefinido,"91,2","12.827,38",N/A,"1,53","1,65 %","4,51 %","8,50 %","16,54 %",...,"0,00 %","0,00 %","0,00 %","0,00 %",0,6426,0,N/A,N/A,N/A
1,AAZQ11,Indefinido,"7,42","384.436,90",86.0,"0,1","1,33 %","4,51 %","8,59 %","18,51 %",...,"1,56 %","0,00 %","0,00 %","0,00 %",0,1944,29.519,N/A,"1,20 % a.a","10,00 % a.a"
2,ABCP11,Shoppings,"83,19","74.205,90",78.0,"0,5","0,59 %","2,48 %","4,94 %","10,54 %",...,"1,08 %","16,48 %","17,73 %","26,11 %",1,2365,14.884,N/A,"0,10 % a.a",N/A
3,AFHI11,Papéis,"91,99","711.943,57",98.0,"1,01","1,11 %","3,44 %","6,78 %","13,48 %",...,"1,16 %","-1,18 %","-0,03 %","6,24 %",14,1169,38.876,N/A,"1,00 % a.a",N/A
4,AGRX11,Indefinido,"7,94","385.095,10",76.0,"0,12","1,53 %","4,67 %","9,14 %","15,55 %",...,"1,15 %","0,00 %","0,00 %","0,00 %",0,3178,18.685,N/A,"1,00 % a.a","10,00 % a.a"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,YUFI11B,Indefinido,nan,N/A,N/A,"0,63","0,00 %","0,00 %","0,00 %","0,00 %",...,"0,60 %","-0,01 %","0,59 %","6,32 %",0,N/A,0,N/A,N/A,N/A
529,ZAGH11,Serviços Financeiros Diversos,"10,22","41.033,95",109.0,"0,06","0,60 %","1,86 %","4,07 %","7,20 %",...,N/A,N/A,N/A,N/A,1,6945,410,N/A,"1,50 % a.a","20,00 % a.a"
530,ZAVC11,Indefinido,"8,6","6.793,52",88.0,"0,13","0,00 %","2,94 %","7,70 %","16,30 %",...,N/A,N/A,N/A,N/A,1,10559,438,N/A,N/A,N/A
531,ZAVI11,Indefinido,"85,0","34.070,95",63.0,"1,1","1,27 %","3,86 %","7,67 %","15,42 %",...,N/A,N/A,N/A,N/A,9,2928,2.751,N/A,"1,25 % a.a","20,00 % a.a"


In [35]:
# dados = []
# dadosTabela = driver.find_element(By.XPATH,'//div/table[contains(@class,"default-fiis-table__container__table")]')

# #print(dadosTabela.text)  

# for linha in dadosTabela.find_elements(By.TAG_NAME,"tr") :
#     linhaDados = []
#     for coluna in linha.find_elements(By.TAG_NAME,"td"):
#         # print(coluna)
#         linhaDados.append(coluna.text)
#     dados.append(linhaDados)


# import pandas as pd
# colunas = ['Fundos','Setor','Preço Atual (R$)','Liquidez Diária (R$)',#
#     'P/VP','Último Dividendo','Dividend Yield','DY (3M) Acumulado',#
#     'DY (6M) Acumulado','DY (12M) Acumulado','DY (3M) média','DY (6M) média',#
#     'DY (12M) média','DY Ano','Variação Preço','Rentab. Período','Rentab. Acumulada',#
#     'Patrimônio Líquido','VPA','P/VPA','DY Patrimonial','Variação Patrimonial',#
#     'Rentab. Patr. Período','Rentab. Patr. Acumulada','Quant. Ativos' ,'Volatilidade' , #
#     'Num. Cotistas' , 'Tax. Gestão' , 'Tax. Performance' , 'Tax. Administração']

# df = pd.DataFrame(dados,columns=colunas)



In [36]:
import os
data_path = str(os.getcwd()) + r"/data/"

In [37]:
data_path

'/home/yair/GHub/Finance-playground/data/'

In [38]:
df.to_csv( data_path+"FE.csv" , sep=";" , decimal="," )

In [39]:
driver.close()

InvalidSessionIdException: Message: Tried to run command without establishing a connection; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
